# 개요

"자연어처리"는, "컴퓨터가 인간의 언어를 알아들을 수 있게 만드는 학문"입니다.

과연 컴퓨터가 인간의 언어를 알아들을 수 있을까요? 2022년 현재 구글과 페이스북, 그리고 각종 대학교와 연구논문의 결과를 보고 있노라면 꿈만 같던 일이 이뤄지고 있는 것 같습니다. 특히 구글이 최근 개발하고 있는 언어모델 AI인 LaMDA(Language Model for Dialog Application) 이슈를 보면 AI가 지각을 가진 것 같은(?) 느낌이 들 정도로 발전했고요.

https://www.mk.co.kr/news/world/view/2022/06/516140/

하지만 모든 최첨단 기술에는 시작이 있는 법. 컴퓨팅 기술과 데이터 가용성이 향상되면서 점차 발전하고 있는 자연어 처리 방식을 1980년대로 거슬러 올라가 순서대로 한 꼬집씩 짚어보고, 비교적 최근 개발된 모델(알고리즘)인 트랜스포머, 어텐션 알고리즘, GPT와 ELMO, BERT 등에 대해서도 가볍게 한 번 알아보겠습니다.


# 1. 기초 - 파이썬으로 텍스트를 다루는 기본 문법

In [28]:
문장 = "동해물과 백두산이 마르고 닳도록 하느님이 보우하사 우리나라 만세"

In [29]:
# "하느님"이 문장 변수 안에 있는지 확인 : True를 리턴
"하느님" in 문장

True

In [30]:
# 문장 내 "백두산"이 출현하는 인덱스 : 5
문장.index("백두산")  # 파이썬 인덱스는 0부터 올라감

5

In [31]:
# "우리나라"는 문장 변수에서 몇 번째 단어인가? : 6번째(인덱스는 0부터)
문장.split().index("우리나라")

6

In [32]:
# 문장 변수의 세 번째 단어(마르고)를 리턴하려면?
문장.split()[2]  # 0, 1, 2

'마르고'

In [33]:
# 문장 변수의 글자 순서를 역순 정렬하여 리턴하려면?
문장[::-1]  # [시작인덱스(생략함) : 끝인덱스(생략함) : 간격(뒤로1칸씩)]

'세만 라나리우 사하우보 이님느하 록도닳 고르마 이산두백 과물해동'

In [34]:
# 문장 변수의 첫 번째 단어와 마지막 단어를 연결하려면?  # "동해물과만세"
단어리스트 = 문장.split()
첫번째단어 = 단어리스트[0]
마지막단어 = 단어리스트[-1]
시작과끝단어 = 첫번째단어 + 마지막단어
print(시작과끝단어)

동해물과만세


In [35]:
# 짝수 인덱스(0, 2, 4, ..)의 단어만 출력하려면?  # 동해물과 마르고 하느님이 우리나라
[단어리스트[i] for i in range(len(단어리스트)) if i % 2 == 0]

['동해물과', '마르고', '하느님이', '우리나라']

In [36]:
# 문장 변수의 마지막 다섯 글자만 출력하려면?  # "나라 만세"
문장[-5:]

'나라 만세'

In [37]:
# 단어 순서는 유지하면서 글자 순서만 역순으로 출력하려면?
" ".join(단어[::-1] for 단어 in 단어리스트)

'과물해동 이산두백 고르마 록도닳 이님느하 사하우보 라나리우 세만'

In [38]:
# 대소문자를 전부 소문자로 바꾸려면?
"Hello World!".lower()  # 대문자는 .upper()

'hello world!'

# 자연어 처리의 시작 : 토큰화

위의 문법이면 웹에서 수집한 텍스트나 DB, 스토리지에서 가져온 텍스트자료를 가지고 기본적인 전처리를 수행하는 데 충분할 것입니다. 이제 "토큰화"부터 본격적인 NLP 단계의 전처리를 시작해봅시다.

토큰화Tokenization는 문장을 구성 단어로 나누는 절차를 말합니다.

### "나는 책을 읽고 있다."

위 문장에서 가장 먼저 처리할 작업은 이 문장의 단어(토큰)를 추출하는 것입니다. 가장 기본적인 방법은 한 단어씩 구분짓는 것입니다. 부호도 포함해서요.

### ["나는", "책을", "읽고", "있다", "."]

이렇게 한 번에 한 단어씩 토큰을 추출했습니다. 이런 경우를 유니그램Unigram이라고 부릅니다. (Uni는 1을 뜻함)

경우에 따라 두 개나 세 개의 토큰을 추출할 수도 있습니다.

### ["나는 책을", "책을 읽고", "읽고 있다", "있다."]

이렇게 두 개씩 묶어 만들면 바이그램Bi-Gram, 세 개씩 묶으면 트라이그램Tri-Gram이라고 부릅니다. 자주는 아니지만 경우에 따라 4개 이상의 단어로 토큰을 추출할 경우 n-그램이라고 부릅니다. (여기서 n은 자연수)

지금까지 설명드린 것이 "단어레벨(Word Level)"의 엔그램이고, 필요에 따라 글자단위Character level) 엔그램으로 토큰을 생성하기도 합니다. 아래처럼요. (캐릭터레벨 토큰화는 대부분 스페이스를 한 개의 글자로 간주합니다.)

### ["나는 ", "는 책", " 책을", "책을 ", "을 읽", " 읽고", "읽고 ", "고 있", " 있다", "있다."]

## n-gram의 활용

왜 엔그램을 알아야 할까요? 자연어처리에서 엔그램은 의외로 많은 곳에 사용됩니다. 자연어처리를 활용하는 몇 가지 애플리케이션이 있는데, 예를 들면 인풋박스에 뭔가 타이핑을 할 때 다음 단어가 무엇이 오는지 예측을 할 수 있겠죠? 오타 같은 것을 발견해서 다른 단어를 추천해줄 수도 있을 겁니다. (현재는 대부분 머신러닝 기반 추천 알고리즘을 사용하고 있습니다.)

## n-gram의 장점

엔그램과 대조되는 또 다른 토큰화 기법으로 Bag-of-Words가 있습니다. 말 그대로 "단어가방"인데요. 바로 다음 순서에 소개드리겠지만, BoW는 단어의 순서가 철저하게 무시됩니다. 엔그램을 쓰면 이 문제를 극복하고 어느 정도 문맥을 유지할 수 있거든요. 여기서 잠깐 BoW의 개념에 대해서도 한 번 짚고 돌아오겠습니다.

## BoW - Bag of Words

아래와 같은 문장이 있다고 생각해보겠습니다.

### "machine learning is fun and is not boring"

이걸 BoW로 만들면 아래처럼 바뀝니다.

![](https://i.ibb.co/5hfBRxq/256.png)

첫 번째 문제는 머신러닝이라는 문맥적인 것을 알고 싶은데 BoW는 머신이 한 개고 러닝이 한 개라고만 알려줍니다.

두 번째 문제는 바로 not의 위치입니다. 주어진 문장은, "머신러닝은 fun하고 boring하지 않다"는 뜻인데, BoW로 넘어가면 is fun and is not boring이나, is not fun and is boring이나 동일한 BoW가 되어버립니다.

이 문장을 단어레벨 바이그램으로 토큰화하면 어떻게 될까요?

### ["machine learning", "learning is", "is fun", "fun and", "and is", "is not", "not boring"]

차이가 보이시나요? 머신러닝이라는 단어가 한 개의 토큰이 되었고, not boring이 하나의 토큰이 되어서 not이 boring과 붙어있다는 걸 컴퓨터가 인식할 수 있겠죠? (어떻게 인식하는지는 조금 뒤에서 다루겠습니다.)

이렇게 엔그램을 활용한 bag of bigram으로 토큰을 만들었을 때 인식률이나 퍼포먼스가 올라갈 것으로 예상할 수 있습니다. 문맥적인 부분을 숫자로 표현할 수 있게 되었으니까요.

엔그램이 활용되는 또 다른 예제는 바로 Next word prediction(다음 단어 예측)입니다. 현재는 굉장히 발전된 단어예측 알고리즘들이 있지만, 예전에는 이런 방식으로도 다소나마 구현이 가능했습니다.

how are you doing, how are you, how are they라는 데이터가 입력되어 있을 때, 트라이그램으로 토큰화를 했다면 대략

### how are you
### are you doing
### how are you
### how are they

이런 네 개의 토큰이 만들어졌을 겁니다. 이를 기반으로 사용자가 검색창에 "how are"를 입력했다면? 이 알고리즘은 카운트 기반으로 "you"를 추천해줄 수 있습니다.

![](https://i.ibb.co/93qpH2C/257.png)

마지막으로 엔그램을 활용하는 또 다른 예로, 스펠체크를 할 수 있을 겁니다.

### quality
### quarter
### quit

이렇게 스펠체커의 knowledge base data에 세 개의 단어로 바이그램이 입력된 상태에서 "qwal"을 입력하면?

![](https://i.ibb.co/4K44sQB/258.png)

qual이라고 추천해줄 수 있겠죠?

이제 토큰화를 파이썬 코드로 간단히 구현해보겠습니다.

In [39]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download("punkt")

words = word_tokenize("We are studying NLP Fundamentals")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smj02\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smj02\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
words

['We', 'are', 'studying', 'NLP', 'Fundamentals']

In [41]:
def generate_N_grams(text, ngram=1):
    "nltk의 불용어 사전 활용한 n그램 함수"
    words = [word for word in text.split(" ") if word not in set(stopwords.words('english'))]
    temp = zip(*[words[i:] for i in range(0, ngram)])
    ans = [' '.join(ngram) for ngram in temp]
    return ans

In [42]:
unigram = generate_N_grams("We are studying NLP Fundamentals")
unigram

['We', 'studying', 'NLP', 'Fundamentals']

In [43]:
bigram = generate_N_grams("We are studying NLP Fundamentals", 2)
bigram

['We studying', 'studying NLP', 'NLP Fundamentals']

# TF-IDF

엔그램이나 BoW를 이용한 NGD 알고리즘은 복잡한 자연어처리에 한계가 있었습니다. 품사를 직접 태깅해야 했고, 불용어를 제거해야 하며, 텍스트 정규화, 철자수정, 어간추출이나 단어의 중의성 문제 등.. 수많은 문제를 해결해야 했습니다. 특히 표제어를 추출하거나문서간 유사도를 측정하는 등의 간단한 연산도 "단어가 몇 번 들어가는가?" 정도의 숫자세기에서 그치게 되었습니다.

이런 문제에 대한 진일보한 해결책들이 하나씩 개발되었는데, TF-IDF도 그 중 하나입니다. TF-IDF는 Term Frequency - Invese Document Frequency의 약자입니다. Term이란 문장이 주어졌을 때 단어를 뜻한다고 보시면 됩니다. 이걸 왜 사용하게 되었을까요? 문서, 즉 문장들은 단어로 구성되어 있는데, 이 단어들을 통해 문서의 연관성을 알고 싶을 때 TF-IDF 알고리즘을 사용합니다. 문장을 구성하는 단어별로 문서에 대한 "정보"를 얼마나 많이 가지고 있을까 하는 것을 "숫자"로 표현할 수 있는 알고리즘입니다.

먼저 알아볼 것은 TF입니다. 문서가 주어졌을 때 단어가 몇 번씩 주어졌는지. 문서 안에 단어가 여러 번 출현했다면 연관성이 높을 것이라는 가정하에 빈도에 의한 점수를 매깁니다.

![](https://i.ibb.co/pZNJkcK/259.png)

car가 가장 빈도가 높고 가장 중요한 단어로 간주해도 될 것입니다. 그런데 TF스코어만 사용하면 치명적인 단점이 있습니다. 바로 "흔한 단어" 때문인데요.

![](https://i.ibb.co/BTtQQjj/260.png)

a와 friend가 공동1등입니다. 문서와 연관성은 깊지 않지만 관사나 대명사 외에도 여러 문서에 자주 출현하는 흔한 단어가 있을 것입니다. 이러한 단어들에 패널티를 주기 위해 IDF라는 개념을 사용하는데요. 공식은 간단합니다.

### Log(Total # of Docs / # of Docs with the term in it)

 로그 안에서 총 문장의 갯수를 단어가 출현한 문장의 갯수로 나눠준 값입니다. 수학적 오류를 피하기 위해 분모에 1을 더해주기도 합니다.

![](https://i.ibb.co/g3RBcZ0/263.png)

TF와 IDF를 곱하면 아래와 같은 결과가 나옵니다.

![](https://i.ibb.co/7K2HBZb/263.png)

문장A에서 TF-IDF값이 가장 높은 단어는 0.13의 car이고,
문장B에서는 0.08로 friend라는 것을 볼 수 있습니다.

문장B에서 가장 핵심이 되는 단어, 문장과 가장 연관성이 높은 단어가 각각 car와 friend라는 의미이기도 합니다.

요약하면 문서, 문장이 주어졌을 때 TF-IDF는 각 단어별로 문장과의 연관성을 수치로 나타낸 값이라고 볼 수 있습니다.

# word2vec

이제 본격적으로 단어를 벡터로 변환하는 알고리즘인 word2vec에 대해 알아보겠습니다.

딥러닝 모델에 텍스트를 넣을 수 없죠. 숫자는 넣을 수 있습니다. 자연어처리의 대상은 텍스트인데, 이를 숫자로 바꾸는 것을 인코딩이라고 합니다.

"thank you"와 "I love you"라는 문장이 주어졌을 때 가장 간단한 방법은 출현 순서대로 숫자를 매기는 방법일 것입니다.

 {thank : 0,
 you   : 1,
 I     : 2,
 love  : 3}

이런 식으로 단순하게요.
하지만 딥러닝에서 많이 쓰이는 방법은 원-핫 인코딩입니다. 독립적인 벡터를 만들어주는 방법이죠.

{thank : [1, 0, 0, 0],
 you   : [0, 1, 0, 0],
 I     : [0, 0, 1, 0],
 love  : [0, 0, 0, 1]}

그런데 이런 방식의 단점은 단어간의 유사도를 알 수 없다는 것입니다. "고맙다"와 "사랑한다"는 "고맙다"와 "미워한다"보다 좀 더 가까워야할텐데, 원핫인코딩으로는 그 정보를 표현할 수가 없습니다. 모든 단어 벡터간의 거리가 같아요. 유클리드거리도 전부 동일하고, 코사인유사도를 구하려고 해도 전부 직교하는 벡터니까요.

이러한 문제 때문에 "임베딩embedding"이라는 개념이 도입되었습니다.

인코딩 대신에 임베딩을 사용함으로써 단어벡터끼리의 유사도를 구할 수가 있게 됩니다. 임베딩은 원핫인코딩보다 오히려 차원도 적으면서 유사도를 갖게 됩니다. (학습방법은 아래에서 설명하겠습니다.)

![](https://i.ibb.co/9szggXh/264.png)

비슷한 단어는 서로 가까이 분포하는 것을 보실 수 있습니다.

word2vec은 여러가지 임베딩 기법 중 하나이고요. 이 유사도 같은 경우에는 비슷한 위치에 있는 단어들을 통해 값을 얻게 됩니다. 비슷한 위치에 있는 단어들을 "이웃"이라고 합니다.

word2vec 데이터를 생성하는 가장 단순한 방법은 바로 이웃 단어를 활용한 skipgram을 통해 학습하는 것입니다. skipgram을 간단히 설명드리겠습니다. 여기부터는 반갑게도 우리에게 익숙한 "머신러닝"과 "학습"개념이 나옵니다.

(계속)

# RNN 기초

# LSTM 기초

# seq2seq와 어텐션 모델

# 트랜스포머(Attention is all you need)

# GPT 이해하기

# BERT 이해하기